**mohammed errakho - adil oussidi**

In [ ]:
!pip install librosa

In [ ]:
import pandas as pd
import os
import librosa

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

import matplotlib.pyplot as plt
from scipy.io import wavfile as wav


from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 


In [ ]:
x_train = pd.read_csv("../input/moroccan-darija-trigger-word-classification-ed-2/train.csv")
test = pd.read_csv("../input/moroccan-darija-trigger-word-classification-ed-2/test.csv")

In [ ]:
import librosa
audio_file_path='../input/moroccan-darija-trigger-word-classification-ed-2/data/train/0mbsexwoja8u4n12v5ql.wav'
audio_file_path_test = '../input/moroccan-darija-trigger-word-classification-ed-2/data/test/01nuflorkm26dw7si39az.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [ ]:
audio_file_path_test

In [ ]:
print(librosa_audio_data)

In [ ]:
# Original audio with 1 channel 
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)

In [ ]:
wave_sample_rate, wave_audio = wav.read(audio_file_path) 

In [ ]:
wave_audio

In [ ]:
# Original audio with 2 channels 
plt.figure(figsize=(12, 4))
plt.plot(wave_audio)

In [ ]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

In [ ]:
audio_dataset_path='../input/moroccan-darija-trigger-word-classification-ed-2/data/train'
metadata=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/train.csv')
metadata.head()

In [ ]:
audio_dataset_path_test='../input/moroccan-darija-trigger-word-classification-ed-2/data/test'
metadata_test=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/test.csv')
metadata_test

In [ ]:
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
extracted_features=[]
for index_num,row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["id"]+'.wav'))
    final_class_labels=row["label"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','label'])
extracted_features_df.head()

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['label'].tolist())

In [ ]:
len(X[0])

In [ ]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
num_labels=y.shape[1]
num_labels

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
audio_dataset_path_test='../input/moroccan-darija-trigger-word-classification-ed-2/data/test'
metadata_test=pd.read_csv('../input/moroccan-darija-trigger-word-classification-ed-2/test.csv')

In [ ]:
def features_test():
    extracted_features=[]
    for index_num,row in metadata_test.iterrows():
        file_name = os.path.join(os.path.abspath(audio_dataset_path_test),str(row["id"]+'.wav'))
        data=features_extractor(file_name)
        extracted_features.append(data)
    return extracted_features

In [ ]:
extracted_features = features_test()
extracted_features

In [ ]:
extracted_features_df_test=pd.DataFrame(extracted_features)
extracted_features_df_test.head()

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy)


In [ ]:
audio_file_path_test

In [ ]:
X_test[1]

In [ ]:
np.array(extracted_features).shape

In [ ]:
test_accuracy2=model.predict(np.array(extracted_features))

In [ ]:
test_accuracy2

In [ ]:
[np.argmax(k) for k in test_accuracy2]

In [ ]:
sub = pd.read_csv("../input/moroccan-darija-trigger-word-classification-ed-2/sample_submission.csv")
sub

In [ ]:
my_submission = pd.DataFrame({'Id': test.id, 'label': predicted})